In [2]:
import pandas as pd
#import torch
import sklearn
from sklearn import preprocessing
import time
import numpy as np

from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

# Load the database, shouldn't take that long. This is all data!

t1 = time.time()
print('Loading database ...')
df = pd.read_hdf('../database/all_data_comp.h5','table')
print('Time to load database:', time.time()-t1)


Loading database ...
Time to load database: 1.8056986331939697


In [3]:
fo_1_3_total = df['FO_day_engine_1_3'].dropna()
fo_2_4_total = df['FO_day_engine_2_4'].dropna()

# this gives the total in tonnes for each day.

# The dataset is not complete overlapping in time with data from both the mass-flow meters and the
# the rest of the data. So we have to manually filter out the time interval which we are interested in.

date_begin = '2014-02-01'
date_end = '2014-12-16'

# Dict of var names we want to use.


var_names = {'ae1_frp':'AE1 FUEL RACK POSIT:1742:mm:Average:900',
             'ae2_frp':'AE2 FUEL RACK POSIT:2742:mm:Average:900',
             'ae3_frp':'AE3 FUEL RACK POSIT:3742:mm:Average:900',
             'ae4_frp':'AE4 FUEL RACK POSIT:4742:mm:Average:900',
             
             'ae1_cac_P':'AE1 CA OTL CAC PRESS:1649:Bar:Average:900',
             'ae2_cac_P':'AE2 CA OTL CAC PRESS:2649:Bar:Average:900',
             'ae3_cac_P':'AE3 CA OTL CAC PRESS:3649:Bar:Average:900',
             'ae4_cac_P':'AE4 CA OTL CAC PRESS:4649:Bar:Average:900',
             
             'ae1_cac_ca':'AE1 EXH CA OUTET 1:1543:  C:Average:900',
             'ae2_cac_ca':'AE2 EXH CA OUTET 1:2543:  C:Average:900',
             'ae3_cac_ca':'AE3 EXH CA OUTET 1:3543:  C:Average:900',
             'ae4_cac_ca':'AE4 EXH CA OUTET 1:4543:  C:Average:900',
             
             'ae1_exh':'AE1 EXH MEAN VALUE:1591:  C:Average:900',
             'ae2_exh':'AE2 EXH MEAN VALUE:2591:  C:Average:900',
             'ae3_exh':'AE3 EXH MEAN VALUE:3591:  C:Average:900',
             'ae4_exh':'AE4 EXH MEAN VALUE:4591:  C:Average:900',
             
             'ae1_fo_P':'AE1 FO INLET PRESS:1603:Bar:Average:900',
             'ae2_fo_P':'AE2 FO INLET PRESS:2603:Bar:Average:900',
             'ae3_fo_P':'AE3 FO INLET PRESS:3603:Bar:Average:900',
             'ae4_fo_P':'AE4 FO INLET PRESS:4603:Bar:Average:900',
             
             'ae1_rpm':'AE1 ENG SPEED:1745:RPM:Average:900',
             'ae2_rpm':'AE2 ENG SPEED:2745:RPM:Average:900',
             'ae3_rpm':'AE3 ENG SPEED:3745:RPM:Average:900',
             'ae4_rpm':'AE4 ENG SPEED:4745:RPM:Average:900',
             
             'me1_frp':'ME1 FUEL RACK POSIT:10005:%:Average:900',
             'me2_frp':'ME2 FUEL RACK POSIT:20005:%:Average:900',
             'me3_frp':'ME3 FUEL RACK POSIT:30005:%:Average:900',
             'me4_frp':'ME4 FUEL RACK POSIT:40005:%:Average:900',
             
             'me1_ca_T':'ME1 CA TEMP COOL OUT:1343:C:Average:900',
             'me2_ca_T':'ME2 CA TEMP COOL OUT:2343:C:Average:900',
             'me3_ca_T':'ME3 CA TEMP COOL OUT:3343:C:Average:900',
             'me4_ca_T':'ME4 CA TEMP COOL OUT:4343:C:Average:900',
             
             'me1_cac_T':'ME1 CHARGE AIR TEMP:1347:C:Average:900',
             'me2_cac_T':'ME2 CHARGE AIR TEMP:2347:C:Average:900',
             'me3_cac_T':'ME3 CHARGE AIR TEMP:3347:C:Average:900',
             'me4_cac_T':'ME4 CHARGE AIR TEMP:4347:C:Average:900',
             
             'me1_exh_T':'ME1 EXH GAS MEAN:1125:C:Average:900',
             'me2_exh_T':'ME2 EXH GAS MEAN:2125:C:Average:900',
             'me3_exh_T':'ME3 EXH GAS MEAN:3125:C:Average:900',
             'me4_exh_T':'ME4 EXH GAS MEAN:4125:C:Average:900',
             
             'me1_rpm':'ME1 ENGINE SPEED:1364:rpm:Average:900',
             'me2_rpm':'ME2 ENGINE SPEED:2364:rpm:Average:900',
             'me3_rpm':'ME3 ENGINE SPEED:3364:rpm:Average:900',
             'me4_rpm':'ME4 ENGINE SPEED:4364:rpm:Average:900',
             
             'fo_booster_13':'FO BOOST 1 CONSUMPT:6165:m3/h:Average:900',
             'fo_booster_24':'FO BOOST 2 CONSUMPT:6166:m3/h:Average:900'}

for names in var_names:
    if var_names[names] in list(df):
        #print(var_names[names])
        pass
    else:
        print('*** VAR MISSING *** ', var_names[names], ' *** VAR MISSING ***')

        
eng_13 = [var_names['ae1_frp'],
          var_names['ae3_frp'],
          var_names['ae1_cac_P'],
          var_names['ae3_cac_P'],
          var_names['ae1_cac_ca'],
          var_names['ae3_cac_ca'],
          var_names['ae1_exh'],
          var_names['ae3_exh'],
          var_names['ae1_fo_P'],
          var_names['ae3_fo_P'],
          var_names['ae1_rpm'],
          var_names['ae3_rpm'],
          var_names['me1_frp'],
          var_names['me3_frp'],
          var_names['me1_ca_T'],
          var_names['me3_ca_T'],
          var_names['me1_cac_T'],
          var_names['me3_cac_T'],
          var_names['me1_exh_T'],
          var_names['me3_exh_T'],
          var_names['me1_rpm'],
          var_names['me3_rpm']]
          
eng_24 = [var_names['ae2_frp'],
          var_names['ae4_frp'],
          var_names['ae2_cac_P'],
          var_names['ae4_cac_P'],
          var_names['ae2_cac_ca'],
          var_names['ae4_cac_ca'],
          var_names['ae2_exh'],
          var_names['ae4_exh'],
          var_names['ae2_fo_P'],
          var_names['ae4_fo_P'],
          var_names['ae2_rpm'],
          var_names['ae4_rpm'],
          var_names['me2_frp'],
          var_names['me4_frp'],
          var_names['me2_ca_T'],
          var_names['me4_ca_T'],
          var_names['me2_cac_T'],
          var_names['me4_cac_T'],
          var_names['me2_exh_T'],
          var_names['me4_exh_T'],
          var_names['me2_rpm'],
          var_names['me4_rpm']]
        

# Create np arrays. X vals are engine inputs, y are fo flows    
# For the first tries I am only using the variables below...

X_13 = np.array(df[[
            var_names['ae1_frp'],
            var_names['ae3_frp'],
            var_names['me1_frp'],
            var_names['me3_frp'],
            var_names['ae1_rpm'],
            var_names['ae3_rpm'],
            var_names['me1_rpm'],
            var_names['me3_rpm'],       
            ]][date_begin:date_end])
X_24 = np.array(df[[
            var_names['ae2_frp'],
            var_names['ae4_frp'],
            var_names['me2_frp'],
            var_names['me4_frp'],
            var_names['ae2_rpm'],
            var_names['ae4_rpm'],
            var_names['me2_rpm'],
            var_names['me4_rpm'],
            ]][date_begin:date_end])

X_13 = np.array(df[eng_13][date_begin:date_end])
X_24 = np.array(df[eng_24][date_begin:date_end])

y_13 = np.array(df[var_names['fo_booster_13']][date_begin:date_end]).reshape(-1,1)
y_24 = np.array(df[var_names['fo_booster_24']][date_begin:date_end]).reshape(-1,1)

X_train_13, X_test_13, y_train_13, y_test_13 = train_test_split(X_13, y_13, train_size=0.75, test_size=0.25)


In [ ]:
#  modeling with tpot


# Number of generations, should be boosted if we want better results. 3 gens take ~ 30min on my macbook...
gen = 2
cores=-1 # use all of them

X_train_13, X_test_13, y_train_13, y_test_13 = train_test_split(X_13, y_13, train_size=0.75, test_size=0.25)
tpot = TPOTRegressor(generations=gen, population_size=50, verbosity=2, n_jobs=cores)
tpot.fit(X_train_13, y_train_13.reshape(-1,))

print(tpot.score(X_test_13,y_test_13))


Optimization Progress:  33%|███▎      | 50/150 [03:11<10:59,  6.59s/pipeline]

In [8]:
tpot

TPOTRegressor(config_dict={'sklearn.linear_model.ElasticNetCV': {'l1_ratio': array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ]), 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}, 'sklearn.ensemble.ExtraT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
       crossover_rate=0.1, cv=5, disable_update_check=False,
       early_stop=None, generations=2, max_eval_time_mins=5,
       max_time_mins=None, mutation_rate=0.9, n_jobs=8, offspring_size=50,
       periodic_checkpoint_folder=None, population_size=50,
       random_state=None, scoring=None, subsample=1.0, verbosity=2,
       warm_start=False)

In [10]:
X_13.shape

(30624, 8)

In [11]:

# Lets just make a quick prediction..

print(tpot.predict(X_13[3].reshape(1,-1)))
print(y_13[5])

# Export the model
tpot.export('eng13.py')

tpot_13 = tpot

# This is how to import the model into another code:
# import eng13
# ... or:
# exec(open('eng13.py').read(), globals())

[ 0.32380799]
[ 0.4179048]


In [12]:
tpot.evaluated_individuals_

{'DecisionTreeRegressor(CombineDFs(input_matrix, input_matrix), DecisionTreeRegressor__max_depth=6, DecisionTreeRegressor__min_samples_leaf=10, DecisionTreeRegressor__min_samples_split=3)': (1,
  -0.0051467999041840823),
 'DecisionTreeRegressor(KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=10, KNeighborsRegressor__p=1, KNeighborsRegressor__weights=uniform), DecisionTreeRegressor__max_depth=8, DecisionTreeRegressor__min_samples_leaf=8, DecisionTreeRegressor__min_samples_split=7)': (2,
  -0.0020375260309837338),
 'DecisionTreeRegressor(SelectPercentile(CombineDFs(input_matrix, input_matrix), SelectPercentile__percentile=52), DecisionTreeRegressor__max_depth=6, DecisionTreeRegressor__min_samples_leaf=10, DecisionTreeRegressor__min_samples_split=3)': (2,
  -0.0063004323898824084),
 'DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=1, DecisionTreeRegressor__min_samples_leaf=5, DecisionTreeRegressor__min_samples_split=11)': (1,
  -0.10805561338217158)

In [8]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator

X_train_13, X_test_13, y_train_13, y_test_13 = train_test_split(X_13, y_13, train_size=0.75, test_size=0.25)

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = y_13
features = X_13
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data, random_state=42)

# Score on the training set was:-0.00013557952884572924
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    StackingEstimator(estimator=LassoLarsCV(normalize=False)),
    ExtraTreesRegressor(bootstrap=False, max_features=0.6500000000000001, min_samples_leaf=4, min_samples_split=17, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target.ravel())
results = exported_pipeline.predict(testing_features)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 299 iterations, alpha=2.206e+01, previous alpha=8.040e-02, with an active set of 98 regressors.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 289 iterations, alpha=3.904e+00, previous alpha=7.850e-02, with an active set of 100 regressors.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 238 iterations, i.e. alpha=1.482e-01, with an active set of 94 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/anaconda/lib/

In [4]:
from sklearn.externals import joblib

In [12]:
joblib.dump(exported_pipeline,'model_10gen.pk1')

['model_10gen.pkg']

In [5]:
model_13_gen10 = joblib.load('model_10gen.pk1')

In [6]:
model_13_gen10.predict(X_13[1])

/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 0.42037233])

In [1]:
100-sklearn.metrics.mean_squared_error(tpot.predict(X_test_13),y_test_13)/(max(y_test_13)-min(y_test_13))*100

NameError: name 'sklearn' is not defined

In [2]:
y_13_predict = model_13_gen10.predict(X_test_13)
print(y_13_predict[5])
print(y_13[5])

NameError: name 'model_13_gen10' is not defined

In [25]:
%pylab qt5

n1=0
n2=500

x=np.linspace(n1,n2,n2-n1)

#plt.scatter(x,y_13[:n2])
#plt.scatter(x,y_13_predict[:n2])

fig, ax = plt.subplots()


plt.plot(x,y_test_13[n1:n2],label='Truth value')
plt.plot(x,y_13_predict[n1:n2], label='Predicted value')

ax.set(xlabel='test no', ylabel='FO flow $m^3/h$',
       title='Model TPOT, 10 generations. Test data.')
#ax.grid()

plt.legend()
plt.show()

Populating the interactive namespace from numpy and matplotlib


In [27]:
%pylab qt5

n1=0
n2=500

x=np.linspace(n1,n2,n2-n1)


fig, ax = plt.subplots()

ax.set(xlabel='Predicted value', ylabel='Truth value',
       title='Model TPOT, 10 generations. Test data.')

#plt.scatter(y_13_predict[n1:n2],y_test_13[n1:n2]) #,label='Truth value')
plt.scatter(y_13_predict,y_test_13,marker='.',label='FO Flow')
#plt.plot(x,y_13_predict[n1:n2], label='Predicted value')
plt.legend()
plt.show()

Populating the interactive namespace from numpy and matplotlib


In [65]:
MSE = sklearn.metrics.mean_squared_error(tpot.predict(X_test_13), y_test_13)
print('Model eng1/3 with TPOT MSE:', MSE)

MSE_perc = sklearn.metrics.mean_squared_error(tpot.predict(X_13),y_13)/(max(y_13)-min(y_13))*100
print('Model eng1/3 with TPOT MSE percent accuracy:', MSE_perc)

Model eng1/3 with TPOT MSE: 0.000321092177567
Model eng1/3 with TPOT MSE percent accuracy: [ 0.00774175]


In [30]:
#  The other engine pair.

# Number of generations, should be boosted if we want better results. 3 gens take ~ 30min on my macbook...
gen = 3

X_train_24, X_test_24, y_train_24, y_test_24 = train_test_split(X_13, y_13, train_size=0.75, test_size=0.25)
tpot_24 = TPOTRegressor(generations=gen, population_size=50, verbosity=2)
tpot_24.fit(X_train_24, y_train_24)

print(tpot_24.score(X_test_24,y_test_24))


/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Optimization Progress:  50%|█████     | 100/200 [10:15<07:12,  4.33s/pipeline] 

Generation 1 - Current best internal CV score: 0.0010504567993377197


Optimization Progress:  75%|███████▌  | 150/200 [15:51<07:06,  8.52s/pipeline]

Generation 2 - Current best internal CV score: 0.0010504567993377197


Generation 3 - Current best internal CV score: 0.0010504567993377197

Best pipeline: XGBRegressor(LassoLarsCV(input_matrix, normalize=False), learning_rate=0.1, max_depth=8, min_child_weight=2, n_estimators=100, nthread=1, subsample=0.45)
0.00105974029125


In [36]:
# Export the model

tpot_24.export('eng24.py')


True

In [35]:

MSE_13 = sklearn.metrics.mean_squared_error(tpot.predict(X_test_13), y_test_13)
print('Model eng1/3 with TPOT MSE:', MSE_13)

MSE_13_perc = 100-sklearn.metrics.mean_squared_error(tpot.predict(X_13),y_13)/(max(y_13)-min(y_13))*100
print('Model eng1/3 with TPOT MSE percent accuracy:', MSE_13_perc)

MSE_24 = sklearn.metrics.mean_squared_error(tpot.predict(X_test_24), y_test_24)
print('Model eng2/4 with TPOT MSE:', MSE_24)

MSE_24_perc = 100-sklearn.metrics.mean_squared_error(tpot.predict(X_24),y_24)/(max(y_24)-min(y_24))*100
print('Model eng2/4 with TPOT MSE percent accuracy:', MSE_24_perc)


Model eng1/3 with TPOT MSE: 0.000912624265516
Model eng1/3 with TPOT MSE percent accuracy: [ 99.98512634]
Model eng2/4 with TPOT MSE: 0.00042016566897
Model eng2/4 with TPOT MSE percent accuracy: [ 99.19915618]


In [9]:
for i in np.random.randint(1,1000,30):
    print('Prediction:', tpot_13.predict(X_13[i].reshape(1,-1)), 'Truth:',y_13[i], 'Percent:', tpot_13.predict(X_13[i].reshape(1,-1))/y_13[i] *100 )
print('MSE: ',sklearn.metrics.mean_squared_error(tpot_13.predict(X_13),y_13))

Prediction: [ 0.61570013] Truth: [ 0.62026672] Percent: [ 99.2637699]
Prediction: [ 0.43884284] Truth: [ 0.44411433] Percent: [ 98.81303322]
Prediction: [ 0.6089502] Truth: [ 0.61104767] Percent: [ 99.65674255]
Prediction: [ 0.59997397] Truth: [ 0.59626672] Percent: [ 100.62174269]
Prediction: [ 0.65149435] Truth: [ 0.65097148] Percent: [ 100.0803214]
Prediction: [ 0.2973034] Truth: [ 0.29386669] Percent: [ 101.16948019]
Prediction: [ 0.52074187] Truth: [ 0.50803813] Percent: [ 102.50054759]
Prediction: [ 0.67944328] Truth: [ 0.67664768] Percent: [ 100.41315433]
Prediction: [ 0.00351207] Truth: [ 0.00342857] Percent: [ 102.43548079]
Prediction: [ 0.34107567] Truth: [ 0.34956194] Percent: [ 97.5723116]
Prediction: [ 0.41574084] Truth: [ 0.41866671] Percent: [ 99.3011455]
Prediction: [ 0.38722333] Truth: [ 0.41622861] Percent: [ 93.03140829]
Prediction: [ 0.00356816] Truth: [ 0.00342857] Percent: [ 104.07145917]
Prediction: [ 0.00354253] Truth: [ 0.00342857] Percent: [ 103.32368538]
Pred

In [43]:
err = np.array(tpot_13.predict(X_13)/y_13 *100)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


In [47]:
import matplotlib.pyplot as plt
plt.plot(err)

KeyboardInterrupt: 

In [48]:
err

array([[  67.32455435,   88.30832434,   91.63937975, ...,  180.06640346,
         217.0309055 ,  216.07497744],
       [  68.27959826,   89.56103709,   92.93934576, ...,  182.62076604,
         220.10963431,  219.14014577],
       [  69.06085764,   90.58579999,   94.00276348, ...,  184.7103241 ,
         222.62814231,  221.64756083],
       ..., 
       [  37.61118448,   49.33386801,   51.19477806, ...,  100.59495803,
         121.245354  ,  120.71132021],
       [  30.98004583,   40.63593085,   42.16874827, ...,   82.85929978,
          99.86887346,   99.4289939 ],
       [  31.52977439,   41.35699924,   42.91701589, ...,   84.32960501,
         101.64100679,  101.19332175]])

In [13]:
import mxnet as mx

In [14]:
a = mx.nd.ones((2,3))

In [15]:
b = a * 2 +1

In [16]:
b.asnumpy()

array([[ 3.,  3.,  3.],
       [ 3.,  3.,  3.]], dtype=float32)

In [19]:
#Training data
train_data = X_13
train_label = np.array([train_data[i][0] + 2 * train_data[i][1] for i in range(100)])
batch_size = 1

eval_data = y_13
eval_label = np.array([11])

In [22]:
train_label

array([ 30.4       ,  29.53333333,  29.6       ,  29.6       ,
        29.26666667,  29.33333333,  29.33333333,  29.46666667,
        29.        ,  28.66666667,  29.26666667,  29.26666667,
        29.53333333,  28.86666667,  29.06666667,  29.33333333,
        29.4       ,  29.73333333,  30.93333333,  30.93333333,
        28.46666667,  28.53333333,  27.66666667,  29.4       ,
        29.4       ,  29.33333333,  17.8       ,  36.06666667,
        48.86666667,  43.66666667,  34.73333333,  32.33333333,
        32.13333333,  31.26666667,  27.53333333,  37.26666667,
        19.26666667,  34.8       ,  33.        ,  33.66666667,
        33.4       ,  32.53333333,  33.26666667,  31.53333333,
        19.46666667,  19.4       ,  19.6       ,  19.66666667,
        19.4       ,  19.46666667,  19.2       ,  19.2       ,
        19.26666667,  19.66666667,  19.66666667,  19.73333333,
        18.86666667,  19.06666667,  19.2       ,  19.06666667,
        19.        ,  41.66666667,  59.8       ,  73.86